# Challenge 2

¿Cuán rápido puedes ordenar estos
productos?

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [182]:
df = pd.read_csv('ordenamiento.csv')

In [3]:
df.head()

,item_id,vertical,category,domain,score
0,512208310,CPG,PETS FOOD,MLC-CATS_AND_DOGS_FOODS,0.0272
1,468513076,CE,ELECTRONICS,MLC-RANGES,0.9256
2,614337410,CE,ELECTRONICS,MLC-SMART_SPEAKERS,0.8304
3,634351318,APP & SPORTS,APPAREL,MLC-PANTS,0.0560
4,528383704,ACC,VEHICULAR MULTIMEDIA,MLC-GPS,0.2334


In [228]:
def order_dataframe(df):
    df = df.sort_values('score', ascending=False).reset_index(drop=True)
    index = df.index.to_numpy()
    items_ids = df['item_id'].to_list()
    indexes_locked = []

    # Rules 1 and 2
    vertical_counter = 0
    domain_counter = 0
    last_vertical = None
    last_domain = None
    for i in range(0, len(index)):
        idx = index[i]
        mask = True
        row = df.loc[idx] 
        if row['vertical'] == last_vertical:
            vertical_counter += 1
        else:
            vertical_counter = 0
        if row['domain'] == last_domain:
            domain_counter += 1
        else:
            domain_counter = 0

        if vertical_counter == 1:
            mask = mask & (df.iloc[i::]['vertical'] != last_vertical)
        if domain_counter == 4:
            mask = mask & (df.iloc[i::]['domain'] != last_domain)
            
        if isinstance(mask, pd.Series):
            new_idx = df.iloc[i::][mask].iloc[0].name
            index = index[index != new_idx]
            index = np.insert(index, i, new_idx)
            idx = new_idx

        last_vertical = df.loc[idx, 'vertical']
        last_domain = df.loc[idx, 'domain']
        df = df.reindex(index)

    df = df.reset_index(drop=True)
    index = df.index.to_numpy()
    # Rule 3
    if 641416750 in items_ids:
        idx = df[df['item_id'] == 641416750].index[0]
        indexes_locked.append(2)
        index = index[index != idx]
        index = np.insert(index, 2, idx)
    # Rule 4
    if 22351223 in items_ids:
        idx = df[df['item_id'] == 22351223].index[0]
        indexes_locked.append(5)
        index = index[index != idx]
        index = np.insert(index, 5, idx)
    # Rule 5
    # Search for the closest sku in the sorted dataframe to fill 9,10,11 positions
    df = df.reindex(index).reset_index(drop=True)
    index = df.index.to_numpy()
    
    indexes_home_decor = df.query("category == 'HOME&DECOR'").index.to_numpy()
    indexes_rule5 = np.array([el for el in indexes_home_decor if el not in indexes_locked])
    indexes_to_insert = []
    # Search for the to the closest HOME&DECOR item for each position
    for i in range(8,11):
        # Calculate distances
        distances = abs(indexes_rule5-i)
        index_min_distance = indexes_rule5[np.argmin(distances)]
        indexes_to_insert.append(index_min_distance)
        # Remove index from the eligible rule 5 list
        indexes_rule5 = indexes_rule5[indexes_rule5!=index_min_distance]
        
    remove_mask = ~np.isin(index, indexes_to_insert)
    index = index[remove_mask]
    index = np.insert(index, 8, indexes_to_insert)
    return df.reindex(index)

In [230]:
%%time
df_treated = order_dataframe(df)

Wall time: 9.02 s
